# Introduction 

Elasticsearch est un moteur de recherche temps réel et Open Source. 
Il possède de nombreux avantages : 
- il met en place une API RESTful.
- il est distribué, ce qui lui permet d'être tolérant aux pannes.
- il est basé sur le moteur d'indexation d'Apache Lucène.
- utilise le format JSON pour le stockage
- permet de faire de la recherche en texte libre. 

Il est utilisé dans de nombreuses entreprises pour faire de la recherche textuelle dans des documents ou alors traiter des tera de logs. 

Dans ce cours nous allons aborder deux technologies, ElasticSearch et Kibana. 

## Concepts

- `Near Realtime (NRT)` : La plupart des anciens moteurs de recherche devaient processer et indexer les documents pour qu'ils puissent être recherchés. ElasticSearch permet de rechercher les nouveaux documents presque instantanément.  


- `Document` : Le document est l'unité de base  qui peut être indexé dans ElasticSearch. Un document est représenté au format JSON. On peut stocker autant de document que l'on souhaite dans un index. Un document est un ensemble de données clés-valeurs.   


- `Index` : Un index est une collection de documents qui ont des caractéristiques similaires. Un index est identifié par un nom. On peut créer autant d'index que l'on veut.   


- `Node` : Un Node (ou noeud en francais) est un serveur qui fait parti d'un cluster de plusieurs noeuds. Un noeud stocke les données et est optimisé pour retrouver les données.   


- `Cluster` : Un cluster est un ensemble de noeud qui communiquent entre eux.  Un cluster peut contenir autant de noeuds que l'on veut.   


- `Shards` :  Un index peut être tellement gros que la donnée des documents est plus importante que la capacité de stockage d'un node et donc d'un serveur. Pour pallier ce problème ElasticSearch met en place une méthode pour découper la données des index en des nombreuses petites parties qui sont appelés des shards. Chaque shard est stocké sur un noeud différents. 

    Cette technique est intéressante car elle permet de scaler horizontalement notre cluster (ie: augmenter le nombre de machines et donc de noeuds)  
    

- `Replicas` : Comme chaque noeud ne contient pas l'intégralité des données, on pourrait penser que si un noeud tombe en panne, une partie des données serait perdue. ElasticSearch met en place une technique de réplication qui permet de stocker ces différents shards sur plusieurs noeuds. C'est la **réplication**.  
    
    Cette technique est particulièrement intéressante puisqu'elle permet de palier la panne de shards ou de noeuds.

## Lancer un container ElasticSearch et Kibana

Ici, nous avons besoin de deux containers docker qui vont devoir communiquer entre eux. Un container pour ElasticSearch et un pour Kibana qui est l'interface graphique. Vous avez pu voir en début d'unité que lorsque vous avez besoin d'orchestrer le démarrage de plusieurs dockers pour constituer une application, il est préférable d'utiliser docker-compose. C'est ce que nous allons donc faire.

Un fichier docker compose est présent dans cette section. À vous de le lancer et de bien vérifier que les containers tournent correctement !

Pour référence, voici la documentation de l'image ElasticSearch : https://hub.docker.com/_/elasticsearch

Et celle de Kibana : https://hub.docker.com/_/kibana


# Let's Play 

In [89]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch("http://localhost:9200")

In [ ]:
es_client.ping()

Pour indexer des documents, il suffit d'appeler la méthode `index` du client.

In [91]:
document = {
    "name":"Decision Trees", 
    "description":"A decision tree is a decision support tool that uses a tree-like graph or model of decisions and their possible consequences, including chance-event outcomes, resource costs, and utility. Take a look at the image to get a sense of how it looks like.",
    "algo_type":"Supervised Learning"
}

In [ ]:
res = es_client.index(index="algorithms", id=1, body=document)
print(res['result'])

Le document à été créé avec succès, on peut maintenant le récupérer via son index. 

In [94]:
import pprint

In [ ]:
res = es_client.get(index="algorithms", id=1)
pprint.pprint(res["_source"])

In [96]:
documents = [
    {
    "name":"Naive Bayes Classification", 
    "description":"Naive Bayes classifiers are a family of simple probabilistic classifiers based on applying Bayes’ theorem with strong (naive) independence assumptions between the features. The featured image is the equation — with P(A|B) is posterior probability, P(B|A) is likelihood, P(A) is class prior probability, and P(B) is predictor prior probability.",
    "algo_type":"Supervised Learning"
    },    {
    "name":"Logistic Regression", 
    "description":"Logistic regression is a powerful statistical way of modeling a binomial outcome with one or more explanatory variables. It measures the relationship between the categorical dependent variable and one or more independent variables by estimating probabilities using a logistic function, which is the cumulative logistic distribution.",
    "algo_type":"Supervised Learning"
},    {
    "name":"Principal Component Analysis", 
    "description":"PCA is a statistical procedure that uses an orthogonal transformation to convert a set of observations of possibly correlated variables into a set of values of linearly uncorrelated variables called principal components.",
    "algo_type":"Unsupervised Learning"
},    {
    "name":"Singular Value Decomposition", 
    "description":"In linear algebra, SVD is a factorization of a real complex matrix. For a given m * n matrix M, there exists a decomposition such that M = UΣV, where U and V are unitary matrices and Σ is a diagonal matrix.",
    "algo_type":"Unsupervised Learning"
}
]

Si on ne précise pas l'ID lors de l'indexation, ElasticSearch se charge d'en trouver un automatiquement

In [ ]:
for doc in documents:
    res = es_client.index(index="algorithms", id=None, body=doc)
    print(res["result"])

On récupère maintenant tous les éléments de l'index

In [ ]:
result = es_client.search(index="algorithms", body={"query": {"match_all": {}}})
result

La réponse nous donne des informations sur le temps d'exécution de la requête le nombre de documents correspondant à la requête et l'ensemble des documents. 

In [ ]:
N_DOCS = result['hits']['total']['value']
f"{N_DOCS} document{'s' if N_DOCS> 1 else '' } correspondent à la requêtes qui a pris {result['took']} ms"

Affichons maintenant tous les documents : 

In [ ]:
ids = []
for hit in result['hits']['hits']:
    print("Name : {name}\n description : {description} \n Type : {algo_type}\n".format(**hit['_source']))
    print("******************")
    ids.append(hit["_id"])

Pour supprimer un document il suffit d'appeler la méthode `delete`.

In [ ]:
es_client.delete(index="algorithms", id=ids[0])

Faisons des choses un peu plus intéressantes maintenant.   

Dans le répertoire `data/` il y a les données de 5000 films de la base de TMDb. 

## Nettoyage

Avant de commencer, il faut nettoyer un peu les données du fichier csv.

In [120]:
import pandas as pd
import json
df_movies = pd.read_csv("./data/tmdb_5000_movies.csv")

In [121]:
def clean_words(l):
    return [elt["name"] for elt in json.loads(l)]

On nettoie alors la donnée des colonnes.

In [122]:
for col in ["genres", "keywords", "production_countries", "spoken_languages"]:
    df_movies.loc[:,col] = df_movies.loc[:,col].apply(clean_words)

Pour indexer les documents, il nous faut une liste de dictionnaires. Pour cela, on peut utiliser la méthode de pandas `to_dict` avec le paramètre `orient=records`.

In [123]:
documents = df_movies.fillna("").to_dict(orient="records")

In [ ]:
documents[0:2]

Pour éviter de faire plein de petits appels à ElasticSearch pour indexer les 5000 films on peut utiliser un helper `bulk` pour indexer tous les documents d'un seul coup et éviter les appels réseaux qui sont très couteux en temps. 

In [125]:
from elasticsearch.helpers import bulk

In [ ]:
def generate_data(documents):
    for docu in documents:
        yield {
            "_index": "movies",
            "_source": {k:v if v else None for k,v in docu.items()},
        }

bulk(es_client, generate_data(documents))

On peut utiliser l'interface http pour vérifier que les index sont bien à jours. L'index `movies` est présent, on peut aussi voir le nombre de documents et la taille en mémoire.  

In [ ]:
!curl "http://localhost:9200/_cat/indices?v"

On peut voir aussi quelques documents avec l'endpoint HTTP suivant

In [ ]:
!curl "http://localhost:9200/movies/_search"

# Kibana 

In [129]:
from IPython.display import Image

C'est le moment de commencer à utiliser Kibana. Kibana est un soft de la suite ELK (ElasticSearch, Kibana, Logstash) qui permet de gérer de façon graphique les données dans les index ElasticSearch.

## Index Pattern

Dans l'onglet `Management` > `Index Management`, vous pourrez trouver des informations concernant vos index. Ici on peut donc voir l'index `movies`

Si vous cliquez sur l'index, dans l'onglet `Mappings`vous pourrez voir tous les champs de l'index ainsi que leur type

In [ ]:
Image("./img/index_pattern.png")

Retournez sur la page d'accueil pour accéder à l'onglet `Discover`accessible depuis `Analytics`

L'interface vous propose alors de créer une `data view`. Donnez un nom à votre data view ainsi qu'un `index pattern` égale à `movies*`. L'index pattern permet de définir quelles données on veut dans notre view, toutes les données respectant dont l'index respecte ce pattern vous faire partir de notre view. La data view va nous permettre de requêter et visualiser nos données.

In [ ]:
Image("./img/data_view.png")

Vous avez donc ensuite une interface avec un apperçu des documents que vous venez d'indexer. Vous pouvez aussi, appliquer des filtres pour faire des requêtes simples sur vos données. Ici, on appliquera un filtre sur le genre `Comedy`.

In [ ]:
Image("./img/discover.png")

Vous pouvez aller ensuite cliquer sur un champ à gauche et appuyer sur `visualize` pour commencer à jouer avec les graphiques. Par exemple, on peut créer un histogram des budgets des différents films. 

In [ ]:
Image("./img/budget.png")

Ou encore des genres des différents films

In [ ]:
Image("./img/gender.png")

Retournons maintenant un peu dans notre code pour commencer à utiliser ses données. Il va maintenant falloir se plonger dans les requêtes ElasticSearch. Ce n'est pas la chose la plus aisée, elles deviennent vite illisibles et complexes. 

In [136]:
QUERY = {
    "query": {
        "match_all": {}
    }
}

Cette requête simple permet de récupérer l'intégralité des documents d'un index.

In [137]:
QUERY = {
  "query": {
    "term" : { 
        "title" : "superman"} 
  }
}

Cette seconde requête permet de récupérer tous les documents dont le titre contient `SuperMan`.

In [ ]:
result = es_client.search(index="movies", body=QUERY)
[elt['_source']['title'] for elt in result["hits"]["hits"]]

On veut maintenant chercher tous les films contenants `SuperMan` mais qui ne contiennent pas `Batman`. Pour cela, on est forcé d'utiliser une requête composée appelée `bool query`  elle permet de transformer chaque requête en un filtre booléen. 

In [139]:
QUERY = {
  "query": {
    "bool" : {
      "must" : {
        "term" : { "title" : "superman" }
      },
      "must_not" : {
                  "term" : { "title" : "batman" }
      }
  }
}
}

In [ ]:
result = es_client.search(index="movies", body=QUERY)
[elt['_source']['title'] for elt in result["hits"]["hits"]]

On peut encore réduire les résultats en filtrant sur le budget du film. On veut en plus des films de SuperMan sans Batman récupérer les films qui ont eu un budget de plus de ou égal à 20M et moins de 55M.

In [141]:
QUERY = {
  "query": {
    "bool" : {
      "must" : [
          {
        "term" : { "title" : "superman" }},
        {"range" : {
          "budget" : { "gte" : 20000000, "lt" : 55000000 }
        }}
      ],
      "must_not" : {
                  "term" : { "title" : "batman" }
      }
  }
}
}

In [ ]:
result = es_client.search(index="movies", body=QUERY)
{elt['_source']['title']:elt['_source']['budget']  for elt in result["hits"]["hits"]}

Si on veut faire une recherche dans plusieurs champs de chaque document de l'index, on peut faire une requête de `multi_match`

In [143]:
QUERY = {
  "query": {
    "multi_match" : {
      "query":    "Smith",
      "fields": [ "title", "overview" ] 
    }
  }
}

In [ ]:
result = es_client.search(index="movies", body=QUERY)
[elt['_source']['title']  for elt in result["hits"]["hits"]]

On peut vouloir trier les résultats selon la popularité par exemple.

In [145]:
QUERY = {
  "query": {
    "multi_match" : {
      "query":    "Smith",
      "fields": [ "title", "overview" ] 
    }
  },
    "sort" : [
        { "popularity" : {"order" : "desc"}}
    ]
}

In [ ]:
result = es_client.search(index="movies", body=QUERY)
{elt['_source']['title']:elt['_source']['popularity']  for elt in result["hits"]["hits"]}

ElasticSearch gère aussi bien les dates. On peut vouloir trier par date de sortie.

In [147]:
QUERY = {
  "query": {
    "multi_match" : {
      "query":    "Smith",
      "fields": [ "title", "overview" ] 
    }
  },
    "sort" : [
        { "release_date" : {"order" : "desc"}}
    ]
}

In [ ]:
result = es_client.search(index="movies", body=QUERY)
{elt['_source']['title']:elt['_source']['release_date']  for elt in result["hits"]["hits"]}

## Aggregations

Les aggregations sont des requêtes complexes, mais qui permettent de faire des opérations très rapides sur les données. 
La syntaxe devient très vite complexe. Par exemple, pour récupérer le nombre de films par genre. 

In [149]:
QUERY = {
    "aggs": {
    "count_gender": {
      "terms": {
        "field": "genres.keyword",
        "size": 5,
        "order": {
          "_count": "desc"
        }
      }
        }
  }
}

In [ ]:
result = es_client.search(index="movies", body=QUERY)
result["aggregations"]["count_gender"]["buckets"]

Le paramètre size permet de récupérer les N premières aggrégations.

Ensuite si on veut aller plus loin on peut vouloir récupérer la moyenne des budgets par genre. Il faut intégrer un nouveau niveau d'aggregation. Pour cela on utilise la syntaxe suivante. 

In [151]:
QUERY = {
    "aggs": {
        "count_gender": {
            "terms": {
                "field": "genres.keyword",
                "size": 3,
                "order": {
                    "average_budget": "desc"
                }
            },
            "aggs": {
                "average_budget":{
                    "avg" : {
                        "field" : "budget"
                    }
                }
            }
        }
    }
}

In [ ]:
result = es_client.search(index="movies", body=QUERY)
result["aggregations"]["count_gender"]["buckets"]

Grâce à sa rapidité dans la recherche de données textuelle, Elastic Search est un bon outil pour créer des moteurs de recherche assez simple. Bien qu'aujourd'hui il existe plein d'outil pour créer de moteur de recherche, Elastic Search reste un bon choix.

# Exercice 

In [153]:
import pandas as pd

In [154]:
df_credits = pd.read_csv("data/tmdb_5000_credits.csv")

In [ ]:
df_credits.head()

1) Nettoyer les données de la DataFrame d'acteurs comme précédement.   
2) Merger les deux DataFrame en utilisant l'identifiant du film.   
3) Créer un nouvel Index `augmented_movies` similaire à l'index précédant, mais en ajoutant les données des acteurs.     
4) Créer une fonction permettant de trouver tous les films d'un acteur.   
5) Trouver quel acteur à jouer dans les films avec les plus gros budgets. 
6) Réaliser, sur Kibana, 3 graphiques de votre choix